In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import numpy as np

def create_grid(study_area_bounds, cell_size=80_000):
    """
    Creates a grid of cells within the specified study area bounds using np.arange for efficiency.
    
    study_area_bounds: tuple containing the bounds of the study area as (minx, miny, maxx, maxy)
    cell_size: size of each grid cell (default is 80 km)
    
    returns: GeoDataFrame containing the grid cells
    """
    minx, miny, maxx, maxy = study_area_bounds
    
    # Generate the range of coordinates using np.arange
    x_coords = np.arange(minx, maxx, cell_size)
    y_coords = np.arange(miny, maxy, cell_size)
    
    # Create the grid cells
    grid_cells = [
        box(x, y, x + cell_size, y + cell_size)
        for x in x_coords
        for y in y_coords
    ]
    
    # Create a GeoDataFrame from the list of cells
    grid_gdf = gpd.GeoDataFrame({'geometry': grid_cells}, crs='EPSG:2154')
    
    return grid_gdf

species = gpd.read_file('/Users/arthurcalvi/Data/species/france_species.shp').to_crs('EPSG:2154')

# Example usage:
study_area_bounds = species.total_bounds  # Define the bounding box of the study area
grid_gdf = create_grid(study_area_bounds)

# Visualize the grid
grid_gdf.plot(edgecolor='black', facecolor='none')

In [ ]:
# Example usage:
phenology_gdf = gpd.read_file('/Users/arthurcalvi/Data/species/france_species.shp').to_crs('EPSG:2154')
#UNARY UNION of phenology : 
unary_union_phenology = gpd.GeoDataFrame(phenology_gdf.unary_union, columns=['geometry'], crs='EPSG:2154')
study_area_bounds = unary_union_phenology.total_bounds 

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
from tqdm import tqdm

def create_grid(study_area_bounds, cell_size):
    minx, miny, maxx, maxy = study_area_bounds
    x_coords = np.arange(minx, maxx, cell_size)
    y_coords = np.arange(miny, maxy, cell_size)
    
    grid_cells = [
        box(x, y, x + cell_size, y + cell_size)
        for x in x_coords
        for y in y_coords
    ]
    
    return gpd.GeoDataFrame({'geometry': grid_cells})

def adaptive_grid_refinement(phenology_gdf: gpd.GeoDataFrame, study_area_bounds, threshold: float = 0.10):
    """
    Refine the grid iteratively based on the intersection percentage with phenology data.
    
    phenology_gdf: GeoDataFrame containing the phenology points
    study_area_bounds: Bounds of the study area as (minx, miny, maxx, maxy)
    threshold: intersection percentage threshold for refinement (default is 10%)
    
    returns: refined grid as a GeoDataFrame
    """
    tiles_sizes = [80_000, 40_000, 20_000, 10_000, 5_000, 2_500, 1_250]
    keep_gdf = gpd.GeoDataFrame(columns=['geometry', 'tile_size', 'intersection_percentage'], crs=phenology_gdf.crs)
    grid_gdf = create_grid(study_area_bounds, tiles_sizes[0])

    for tile_size in tiles_sizes:
        
        # Visualization step (optional)
        fig, ax = plt.subplots(figsize=(7, 7))
        grid_gdf.plot(ax=ax, color='white', edgecolor='red', markersize=1)
        ax.set_title(f"Tile size: {tile_size / 1000} km")
        ax.grid(True)
        plt.show()
        refined_rows = []
        
        for row in tqdm(grid_gdf.itertuples(index=False), total=len(grid_gdf)):
            if phenology_gdf.intersects(row.geometry).any():
                if tile_size > 10_000:
                    refined_rows.append({'geometry': row.geometry, 'tile_size': tile_size, 'intersection_percentage': None})
                else:
                    intersection = phenology_gdf.intersection(row.geometry)
                    intersection_pixels = intersection.area / (10 * 10)
                    intersection_percentage = intersection_pixels / (tile_size**2)
                    refined_rows.append({'geometry': row.geometry, 'tile_size': tile_size, 'intersection_percentage': intersection_percentage})

        refined_gdf = gpd.GeoDataFrame(refined_rows, crs=phenology_gdf.crs)

        keep_gdf = pd.concat([
            keep_gdf,
            refined_gdf[(refined_gdf['intersection_percentage'] >= threshold) | (refined_gdf['tile_size'] <= 1_500)]
        ])

        grid_gdf = refined_gdf[refined_gdf['intersection_percentage'] < threshold]

    return keep_gdf


In [ ]:
refined_grid_gdf = adaptive_grid_refinement(unary_union_phenology, study_area_bounds)